# change courseNo and url each time

In [9]:
no = 2
url = 'https://www.gotokeep.com/plans/570cc63e251678ecaca61891'
courseNo = "course" + str(no)

# fetch data from url
## run only once

In [10]:
from lxml import etree, html
import requests

page = requests.get(url)
tree = html.fromstring(page.content)
#fetch data for current course
print ("begin fetching course" + str(no))
course = {}
id = no
title = tree.xpath('/html/body/div[2]/div[1]/div[1]/div[1]/div/text()')[0]
detail = tree.xpath('/html/body/div[2]/div[1]/div[1]/div[1]/p[1]/text()')[0]
image = tree.xpath('/html/body/div[2]/div[1]/div[1]/@style')[0][21:-1]
ctype = tree.xpath('/html/body/div[2]/div[1]/div[1]/div[1]/p[3]/span/text()')[0]
#fetch data for all exercises
exercises = {}
exercise_hrefs = tree.xpath('/html/body/div[2]/div[1]/div[2]/div[2]/div/a/@href')
for i in range(len(exercise_hrefs)):
    print ("begin fetching exercise" + str(i))
    page = requests.get('https://www.gotokeep.com' + exercise_hrefs[i])
    tree = html.fromstring(page.content)
    #begin fetch data
    image = tree.xpath('/html/body/div[2]/div/div[2]/div/@style')[0][22:-2] 
    name = tree.xpath('/html/body/div[2]/div/div[3]/h2/text()')[0] 
    steps = list(enumerate(tree.xpath('//ul[1]/li/text()')))
    breathes = list(enumerate(tree.xpath('//ul[2]/li/text()')))
    movementfeelings = list(enumerate(tree.xpath('//ul[3]/li/text()')))
    movementfeelingPics = list(enumerate(tree.xpath('//p/img/@src')))
    commonmistakes = list(enumerate(tree.xpath('//ul[4]/li/text()')))
    video = tree.xpath('//div[@class="video-wrapper square-wrapper"]/@data-src')[0]
    #add data into exercise
    exercise = {"image" : image, "name" : name, "steps" : steps
              , "breathes" : breathes, "movementfeelings" : movementfeelings
             , "movementfeelingPics" : movementfeelingPics, "commonmistakes" : commonmistakes
             , "video" : video}
    #add exercise into exercises
    exercises[i] = exercise
course = {"meta" : {"id" : id, "title" : title, "detail" : detail, "image" : image, "ctype" : ctype }, "exercises" : exercises}
print(course)



begin fetching course2
begin fetching exercise0
begin fetching exercise1
begin fetching exercise2
begin fetching exercise3
begin fetching exercise4
begin fetching exercise5
begin fetching exercise6
begin fetching exercise7
begin fetching exercise8
begin fetching exercise9
begin fetching exercise10
begin fetching exercise11
begin fetching exercise12
begin fetching exercise13
begin fetching exercise14
begin fetching exercise15
begin fetching exercise16
begin fetching exercise17
begin fetching exercise18
begin fetching exercise19
{'meta': {'id': 2, 'title': '核心功能入门', 'detail': '核心强弱不仅仅决定腹肌形态，更决定你的运动能力。核心区域上到膈肌，下到盆底肌，包括腹横肌、多裂肌，以及外部的腹直肌、腹内外斜肌等等，由于没有骨骼的直接支撑，一直都是我们人体最容易薄弱的区域。在真实的运动中，我们的核心区域主要的任务是不发生形变，保持稳定的支撑，为我们的运动提供一个中心支点，所以，一般的腹肌训练只是在训练肌肉，而不是具有功能性的核心训练，让我们在这个课程中，一起体会核心发力的感觉', 'image': 'https://static1.gotokeep.com/picture/frame/1500627140082.jpg', 'ctype': 'K1'}, 'exercises': {0: {'image': 'https://static1.gotokeep.com/picture/frame/1500866394531.jpg', 'name': '猫式伸展', 'steps': [(0, '俯撑在瑜伽垫

# download all videos and images to local
## run only once

In [11]:
import urllib.request
import os

if not os.path.exists("./videos/course" + str(no) + "/"):
    os.makedirs("./videos/course" + str(no) + "/")
if not os.path.exists("./images/course" + str(no) + "/"):
    os.makedirs("./images/course" + str(no) + "/")
for index in range(len(exercises)):
    video = exercises[index]["video"]
    image = exercises[index]["image"]
    path1 = "./videos/course" + str(no) + "/" + str(index) + ".mp4"
    path2 = "./images/course" + str(no) + "/" + str(index) + ".png"
    urllib.request.urlretrieve(video,path1)
    urllib.request.urlretrieve(image,path2)

# test google cloud connection and upload data(video and image)
## run only once

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
    

In [ ]:
from google.cloud import storage
import os
#set environment variavle for authentification
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/cxr72/Documents/Python Scripts/Quickstart-03dd286a74fc.json"

storage_client = storage.Client()
buckets = list(storage_client.list_buckets())
print(buckets)

for i in range(len(exercises)):
    videoPath = "./videos/" + str(i) + ".mp4"
    videoBlobName = courseNo + "/videos/" + str(i) + ".mp4"
    imagePath = "./images/" + str(i) + ".png"
    imageBlobName = courseNo + "/images/" + str(i) + ".png"
    upload_blob("quickstart-1592333099507.appspot.com",videoPath, videoBlobName)
    upload_blob("quickstart-1592333099507.appspot.com",imagePath, imageBlobName)
    

# update exercises using url from cloud

In [14]:
for i in range(len(exercises)):
    exercises[i]["video"] = "gs://quickstart-1592333099507.appspot.com/" + courseNo + "/videos/" + str(i) + ".mp4"
    exercises[i]["image"] = "gs://quickstart-1592333099507.appspot.com/" + courseNo + "/images/" + str(i) + ".png"
course["exercises"] = exercises
print(course)

{'meta': {'id': 1, 'title': '瘦腿训练', 'detail': '女神必备，「瘦腿」修炼秘籍！本课程进行了为期 4 周的效果测试，一共 183 人参加。4 周后瘦腿成功率高达 69.7%，平均每人瘦了 2.19cm，相当于牛仔裤小了 2 个码，效果显著。如果配合减脂训练、饮食控制，还能让瘦腿效果更明显。', 'image': 'https://static1.gotokeep.com/picture/frame/1501207648307.jpg', 'ctype': 'K2'}, 'exercises': {0: {'image': 'gs://quickstart-1592333099507.appspot.com/course1/images/0.png', 'name': '侧卧左侧提膝', 'steps': [(0, '侧卧在瑜伽垫上，左腿伸直，微微抬起'), (1, '臀部外侧发力提膝，提膝时大腿尽可能贴近腹部'), (2, '提膝至最高点后膝盖轻点地，然后回到起始状态')], 'breathes': [(0, '提膝时呼气，还原时吸气')], 'movementfeelings': [(0, '臀部外侧有明显收缩感')], 'movementfeelingPics': [(0, 'http://static1.keepcdn.com/picture/exerciseimages/v2/55cc42f6ce780bdc3fc33b48_female_front?v=202010230952'), (1, 'http://static1.keepcdn.com/picture/exerciseimages/v2/55cc42f6ce780bdc3fc33b48_female_back?v=202010230952')], 'commonmistakes': [(0, '错误：提膝时膝盖触地'), (1, '解决：左腿全程悬空')], 'video': 'gs://quickstart-1592333099507.appspot.com/course1/videos/0.mp4'}, 1: {'image': 'gs://quickstart-1592333099507.appspot.com/course1/images/1.p

# export to csv and json 

In [15]:
import json

target = json.dumps(course, ensure_ascii=False)
file = open("course" + str(no) + ".txt", "w+", encoding='utf-8')
file.write(target)


8900

In [ ]:
#test download
from google.cloud import storage


def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # source_blob_name = "storage-object-name"
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    )
download_blob("quickstart-1592333099507.appspot.com","course1/images/0.png","teset.png")